In [ ]:
! python -m spacy download en_core_web_lg

In [4]:
import base64

In [5]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [6]:
import re
import spacy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json

In [7]:
#Function to extract names from the string using spacy
def extract_name(string):
    names = []
#     nlp = spacy.load('en')
    nlp = spacy.load('en_core_web_lg')
    doc = nlp(string)
    ent_list = ['PERSON']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            names.append(ent.text)
    return np.unique(names).tolist()

In [8]:
def extract_org(string):
    org = []
    nlp = spacy.load('en_core_web_lg')
    doc = nlp(string)
    ent_list = ['GPE','NORP','ORG','LOC','PRODUCT']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            org.append(ent.text)
    return np.unique(org).tolist()

In [9]:
#Function to extract Phone Numbers from string using regular expressions
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = np.unique(r.findall(string))
    return [re.sub(r'\D', '', number) for number in phone_numbers]

In [10]:
#Function to extract Email address from a string using regular expressions
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return np.unique(r.findall(string)).tolist()
    #return r.findall(string)

In [11]:
def get_nouns(text):
    nlp = spacy.load('en_core_web_lg')
    doc = nlp(text)
    nouns_list=[]
    for token in doc:
        if token.pos_ == 'PROPN':
            nouns_list.append(token.text)
#     print(re.sub(' +', ' ', ' '.join(nouns_list)))
    return re.sub(' +', ' ', ' '.join(nouns_list))

In [12]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


In [13]:
def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'])

In [14]:
main()

Labels:
Junk
CATEGORY_FORUMS
CATEGORY_PERSONAL
CATEGORY_SOCIAL
IMPORTANT
CATEGORY_UPDATES
CHAT
SENT
INBOX
TRASH
CATEGORY_PROMOTIONS
DRAFT
SPAM
STARRED
UNREAD


In [15]:
with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

In [16]:
if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

In [17]:
service = build('gmail', 'v1', credentials=creds)

In [18]:
service.users().messages().list(userId='me').execute()

{'messages': [{'id': '16ae05eee87c8bc8', 'threadId': '16ae05eee87c8bc8'},
  {'id': '16ae047cafa9a0a7', 'threadId': '16ae047cafa9a0a7'},
  {'id': '16ae022b62b5d75e', 'threadId': '16ace29e20077442'},
  {'id': '16ae000d37896402', 'threadId': '16ae000d37896402'},
  {'id': '16adffbeef111316', 'threadId': '16adffbeef111316'},
  {'id': '16adff4270cebdf9', 'threadId': '16adff4270cebdf9'},
  {'id': '16adfe6f03b35d39', 'threadId': '16adfe6f03b35d39'},
  {'id': '16adfdb302fcb62c', 'threadId': '16adfdb302fcb62c'},
  {'id': '16adfdaafbca0227', 'threadId': '16adfdaafbca0227'},
  {'id': '16adfc268fdf17df', 'threadId': '16adfc268fdf17df'},
  {'id': '16adfb0cfc37515b', 'threadId': '16adfb0cfc37515b'},
  {'id': '16adf8d222a67c60', 'threadId': '16adf8d222a67c60'},
  {'id': '16adf86ea2cd643c', 'threadId': '16adf86ea2cd643c'},
  {'id': '16adf80bbd20aa4f', 'threadId': '16adf80bbd20aa4f'},
  {'id': '16adf721533eed52', 'threadId': '16adf721533eed52'},
  {'id': '16adf4fa70608d62', 'threadId': '16adf4fa70608d62

In [19]:
msgs = service.users().messages().list(userId='me').execute()

In [ ]:
next_page = service.users().messages().list_next(service.users().messages().list(userId='me'), msgs)
next_page.execute()

{'From': 'Shoumik Goswami <shoumikgoswami@gmail.com>', 'To': 'Diljeet Singh <diljeets1994@gmail.com>', 'Subject': 'Re: Test Mail', 'Date': 'Sun, 19 May 2019 11:28:32 +0530'}

In [ ]:
def response_extraction(email_data):
    response = {}
    headers_data = {'From':'', 'To':'', 'Subject':'', 'Date':'', 'Cc':}
#     headers = ['From', 'To', 'Subject', 'Date', 'Cc']
    for data in email_data['payload']['headers']:
        if data['name'] in headers_data:
            headers_data[data['name']] = data['value']
    response['Subject'] = headers_data['Subject']
    response['Date'] = headers_data['Date']
    

In [ ]:
def extract_body(email_data):
    try:
        if ((email_data['payload']['body']['size']==0) and ('parts' in email_data['payload']['parts'][0])):
            for body_data in email_data['payload']['parts'][0]['parts']:
                if body_data['mimeType'] == 'text/html':
                    print("body_data in email_data['payload']['parts'][0]['parts']")
        elif ((email_data['payload']['body']['size']==0) and ('data' in email_data['payload']['parts'][0]['body'])):
            for body_data in email_data['payload']['parts']:
                if body_data['mimeType'] == 'text/html':
                    print("email_data['payload']['parts']")
        elif ((email_data['payload']['body']['size']==0) and ('attachmentId' in email_data['payload']['parts'][0]['body'])):
            pass
        else:
            if email_data['payload']['mimeType'] == 'text/html':
                print("email_data['payload']['body']['data']")
    except Exception as e:
        print(e)

In [21]:
# cleaning text and returning meaningful tokens
import string
import unicodedata
from nltk.corpus import stopwords

def text_cleaning(text):
#     text = [word for word in text.split(' ') if word.lower() not in stopwords.words('english')]
#     text =''.join(nopunc)
    
    # removing non unicode
    text = unicodedata.normalize("NFKD", text)
    #Removing commas and spaces in the resume for an effecient check
#     text = text.replace(',',' ')
#     text = text.replace('\\n',' ')
#     text = text.replace('\\r',' ')
#     text = text.replace('\\r\\n',' ')
#     text = text.replace('*','')
    
    bad_characters = ['\r', '\n', '\\r\\n', ',', '*', ';']
    nopunc = [" " if char in bad_characters else char for char in text]
    text =''.join(nopunc)
    text = re.sub(' +', ' ', text)
    bad_words= ['&gt;', '&gt', '&lt']
    text = text.split(' ')
    text = ['' if word in bad_words else word for word in text]
    text =' '.join(text)
#     # removing punctuation
#     nopunc = [char for char in text if char not in string.punctuation]
#     text =''.join(nopunc)
    # removing stop words
    
    return text

16ad4dd20e0cc588 - courtney


In [ ]:
service.users().messages().list(userId='me').execute()

In [ ]:
for msg in msgs['messages']:
    print(msg['id'])
    email_data = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
    email_data['payload']
    extract_body(email_data)
    

In [ ]:
email_data = service.users().messages().get(userId='me', id='16ae022b62b5d75e', format='full').execute()
email_data['payload']
# [0]['parts'][0]['body']['data']
# ['parts'][0]['body']
soup = BeautifulSoup(base64.urlsafe_b64decode(email_data['payload']['parts'][1]['body']['data']), 'html.parser')
email_text = soup.get_text()
email_text
# email_text = soup.decode_contents()


In [ ]:
email_data

In [ ]:
data = text_cleaning(email_text)
data

In [ ]:
extract_email_addresses(email_text)

In [ ]:
extract_email_addresses(data)

In [ ]:
extract_name(data)

In [ ]:
extract_name(email_text)

In [ ]:
extract_org(email_text)

In [ ]:
extract_org(data)

In [ ]:
extract_phone_numbers(data)

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(email_text)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
response = {'From':'', 'To':'', 'Subject':'', 'Date':''}
for data in email_data['payload']['headers']:
    if data['name'] in response:
        response[data['name']] = data['value']
response

In [ ]:
for msg_id in msgs['messages']:
    response_data = {'From':'', 'To':'', 'Subject':'', 'Date':''}
    email_data = service.users().messages().get(userId='me', id=msg_id['id'], format='full').execute()
    for data in email_data['payload']['headers']:
        if data['name'] in response_data:
            response_data[data['name']] = data['value']
    print()
    print(response_data)
    

In [ ]:
import base64
counter = 0
try:
    for msg_id in msgs['messages']:
        try:
            email_data = service.users().messages().get(userId='me', id=msg_id['id'], format='full').execute()
            if ((email_data['payload']['body']['size']==0) and ('parts' in email_data['payload']['parts'][0])):
                print(msg_id['id'])
                for body_data in email_data['payload']['parts'][0]['parts']:
                    base64.urlsafe_b64decode(body_data['body']['data'])
            elif ((email_data['payload']['body']['size']==0) and ('data' in email_data['payload']['parts'][0]['body'])):
                print(msg_id['id'])
                for body_data in email_data['payload']['parts']:
                    base64.urlsafe_b64decode(body_data['body']['data'])
            elif ((email_data['payload']['body']['size']==0) and ('attachmentId' in email_data['payload']['parts'][0]['body'])):
                print(msg_id['id'])
                for body_data in email_data['payload']['parts']:
                    base64.urlsafe_b64decode(body_data['body']['attachmentId'])
            else:
                print(msg_id['id'])
                base64.urlsafe_b64decode(body_data['body']['data'])
        except Exception as e:
            print(e)
except Exception as e:
    print(e)
        
        

In [ ]:
email_data = service.users().messages().get(userId='me', id='16ac694f2e874497', format='full').execute()
print(email_data.keys())
email_data['payload']['parts'][0]['parts'][0]['body']['data']

In [ ]:
base64.urlsafe_b64decode(email_data['payload']['parts'][0]['parts'][0]['body']['data'])

# Response

{
	"error": "string",
	"contacts": [{
		"firstName": "string",
		"lastName": "string",
		"emailAddress": "string",
		"emailAddressDomain": "string",
		"emailAddressDomainWithoutTLD": "string",
		"phoneNumber": "string",
		"mobilePhone": "string",
		"voipPhone": "string",
		"officePhone": "string",
		"fax": "string",
		"address": "string",
		"title": "string",
		"twitterUrl": "string",
		"twitterHandle": "string",
		"linkedInUrl": "string",
		"linkedInHandle": "string",
		"companyName": "string",
		"website": "string"
	}],
	"isSpammyLookingEmailMessage": true,
	"isSpammyLookingSender": true,
	"isSpam": true,
	"from_LastName": "string",
	"from_FirstName": "string",
	"from_Fax": "string",
	"from_Phone": "string",
	"from_Address": "string",
	"from_Title": "string",
	"from_MobilePhone": "string",
	"from_OfficePhone": "string",
	"from_LinkedInUrl": "string",
	"from_TwitterUrl": "string",
	"from_TwitterHandle": "string",
	"from_EmailAddress": "string",
	"emails": [{
		"from_EmailAddress": "string",
		"from_Name": "string",
		"textBody": "string",
		"htmlLines": ["string"],
		"date": "2019-05-05T22:27:56.124Z",
		"didParseCorrectly": true,
		"to": [{
			"name": "string",
			"emailAddress": "string"
		}],
		"cc": [{
			"name": "string",
			"emailAddress": "string"
		}],
		"htmlBody": "string",
		"spammyLookingEmail": true,
		"subject": "string",
		"cleanedBodyHtml": "string",
		"cleanedBodyPlain": "string"
	}],
	"from_LinkedInHandle": "string",
	"duration": 0,
	"cleanedemailbody": "string",
	"cleanedemailbody_ishtml": true,
	"cleanedemailbody_plain": "string",
	"from_CompanyName": "string",
	"from_Website": "string",
	"from_EmailAddressDomain": "string",
	"from_EmailAddressDomainWithoutTLD": "string"
}

## Logic to get names and emails that belong to each other

In [22]:
data = "---------- Forwarded message ---------\r\nFrom: Courtenay Farquharson <courtenay@parserr.com>\r\nDate: Thu, Mar 14, 2019 at 9:17 AM\r\nSubject: Re: Parserr\r\nTo: Steve Burchett <steve.burchett@silverlinecrm.com>\r\n\r\n\r\nHi Steve,\r\n\r\nThanks for being so patient with us.\r\n\r\nCould I ask if that custom object is definitely writeable by the user you\r\nare authenticating with?\r\n\r\nEngineers are working on making that object available now.\r\n\r\n\r\n\r\n[image: Mailtrack]\r\n<https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&>\r\nSender\r\nnotified by\r\nMailtrack\r\n<https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&>\r\n03/14/19,\r\n9:15:25 AM\r\n\r\nOn Thu, Mar 14, 2019 at 12:49 AM Steve Burchett <\r\nsteve.burchett@silverlinecrm.com> wrote:\r\n\r\n> I tried mapping to the case and account objects to see if I could simply\r\n> create any record.  I received the following error message:\r\n>\r\n> [image: Screen Shot 2019-03-13 at 6.47.27 AM.png]\r\n>\r\n>\r\n> ---------- Forwarded message ---------\r\n> From: Steve Burchett <steve.burchett@silverlinecrm.com>\r\n> Date: Wed, Mar 13, 2019 at 6:19 AM\r\n> Subject: Re: Parserr\r\n> To: Courtenay Farquharson <courtenay@parserr.com>\r\n>\r\n>\r\n> Courtenay,\r\n>\r\n> I am now able to authenticate to the salesforce org; however, I can only\r\n> see a short list of standard objects as depicted below:\r\n>\r\n> [image: Screen Shot 2019-03-13 at 6.14.47 AM.png]\r\n>\r\n> For my demo, I need to create records on a custom object named\r\n> Inbound_Email__c.  Is it possible to provide access to this custom object?\r\n>\r\n> Thanks,\r\n> Steve\r\n>\r\n>\r\n> On Wed, Mar 13, 2019 at 4:27 AM Courtenay Farquharson <\r\n> courtenay@parserr.com> wrote:\r\n>\r\n>> Hi Steve,\r\n>>\r\n>> Thanks for your time today. We have enabled the feature for your account\r\n>> so I believe you SHOULD be able to connect now. Let me know if you can't!\r\n>>\r\n>> --\r\n>> Regards,\r\n>>\r\n>> Courtenay\r\n>>\r\n>> Want to chat? Sure! Book me here <https://calendly.com/courtenay-parserr>\r\n>>\r\n>> [image: Mailtrack]\r\n>> <https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&> Sender\r\n>> notified by\r\n>> Mailtrack\r\n>> <https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&> 03/13/19,\r\n>> 10:25:43 PM\r\n>>\r\n>\r\n>\r\n> --\r\n> Steve Burchett\r\n> Sr. Director - Sales Architecture | Silverline\r\n> steve.burchett@silverlinecrm.com <http://silverlinecrm.com/>\r\n> work: +1 360 909-9407\r\n>\r\n>\r\n> <https://silverline.sigstr.net/uc/57dab42d5df0643221d18af2>\r\n>\r\n>\r\n> --\r\n> Steve Burchett\r\n> Sr. Director - Sales Architecture | Silverline\r\n> steve.burchett@silverlinecrm.com <http://silverlinecrm.com/>\r\n> work: +1 360 909-9407\r\n>\r\n>\r\n> <https://silverline.sigstr.net/uc/57dab42d5df0643221d18af2>\r\n>\r\n\r\n\r\n-- \r\nRegards,\r\n\r\nCourtenay\r\n\r\nWant to chat? Sure! Book me here <https://calendly.com/courtenay-parserr>\r\n\r\n\r\n-- \r\nRegards,\r\n\r\nCourtenay\r\n\r\nWant to chat? Sure! Book me here <https://calendly.com/courtenay-parserr>\r\n"
data = text_cleaning(data)
data

"---------- Forwarded message --------- From: Courtenay Farquharson <courtenay@parserr.com> Date: Thu Mar 14 2019 at 9:17 AM Subject: Re: Parserr To: Steve Burchett <steve.burchett@silverlinecrm.com> Hi Steve Thanks for being so patient with us. Could I ask if that custom object is definitely writeable by the user you are authenticating with? Engineers are working on making that object available now. [image: Mailtrack] <https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&> Sender notified by Mailtrack <https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&> 03/14/19 9:15:25 AM On Thu Mar 14 2019 at 12:49 AM Steve Burchett < steve.burchett@silverlinecrm.com> wrote: > I tried mapping to the case and account objects to see if I could simply > create any record. I received the following error message: > > [image: Screen Shot 2019-03-13 at 6.47.27 AM.png] > > > ---------- Forwarded message --------- > From: Steve Bu

In [40]:
data.split('>')

['---------- Forwarded message --------- From: Courtenay Farquharson <courtenay@parserr.com',
 ' Date: Thu Mar 14 2019 at 9:17 AM Subject: Re: Parserr To: Steve Burchett <steve.burchett@silverlinecrm.com',
 ' Hi Steve Thanks for being so patient with us. Could I ask if that custom object is definitely writeable by the user you are authenticating with? Engineers are working on making that object available now. [image: Mailtrack] <https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&',
 ' Sender notified by Mailtrack <https://mailtrack.io?utm_source=gmail&utm_medium=signature&utm_campaign=signaturevirality5&',
 ' 03/14/19 9:15:25 AM On Thu Mar 14 2019 at 12:49 AM Steve Burchett < steve.burchett@silverlinecrm.com',
 ' wrote: ',
 ' I tried mapping to the case and account objects to see if I could simply ',
 ' create any record. I received the following error message: ',
 ' ',
 ' [image: Screen Shot 2019-03-13 at 6.47.27 AM.png] ',
 ' ',
 ' ',
 ' ------

In [25]:
import operator

In [37]:
def find_correct_name(names, email_addr):
    name_counter={name.title():0 for name in names}
    for name in name_counter:
        for item in name.split(' '):
            name_counter[name] += len(re.findall(item.lower(), email_addr))
    print(name_counter)
    name_counter = sorted(name_counter.items(), key=operator.itemgetter(1), reverse= True)
    if name_counter[0][1]>0:
        return name_counter[0][0]
#     return max(name_counter.items(), key=operator.itemgetter(1))[0]

In [45]:
for line in data.split('>'):
    email_addrs = extract_email_addresses(line)
    name=None
    if len(email_addrs) == 1:
        names = extract_name(line)
        if len(names)>0:
            name= find_correct_name(names, email_addrs[0])
#         elif len(names) == 1:
#             name = names[0]
        else:
            pass
    else:
        pass
    if name is not None:
        print([name, email_addrs[0]])
        print()

{'Courtenay Farquharson': 1}
['Courtenay Farquharson', 'courtenay@parserr.com']

{'Parserr': 0, 'Steve Burchett': 2}
['Steve Burchett', 'steve.burchett@silverlinecrm.com']

{'Steve Burchett': 2}
['Steve Burchett', 'steve.burchett@silverlinecrm.com']

{'Steve Burchett': 2}
['Steve Burchett', 'steve.burchett@silverlinecrm.com']

{'Courtenay Farquharson': 1}
['Courtenay Farquharson', 'courtenay@parserr.com']



Testing 

In [42]:
names = ['Parserr', 'Courtenay Farquharson']
email_addr='steve.burchett@silverlinecrm.com'
name_counter={name.title():0 for name in names}
print(name_counter)
for name in name_counter:
    for item in name.split(' '):
        name_counter[name] += len(re.findall(item.lower(), email_addr))

name_counter = sorted(name_counter.items(), key=operator.itemgetter(1), reverse= True)
print(name_counter)
if name_counter[0][1]>0:
        print(name_counter[0][0])
#     return max(name_counter.items(), key=operator.itemgetter(1))[0]

{'Parserr': 0, 'Courtenay Farquharson': 0}
[('Parserr', 0), ('Courtenay Farquharson', 0)]
